### Mega Parse

In [ ]:
from pathlib import Path
import sys
sys.path.append('..')
from megaparse.converter import MegaParse
import os 

api_key: str | None = os.getenv("LLAMA_CLOUD_API_KEY")
print(api_key)

converter = MegaParse(file_path="../megaparse/tests/input_tests/MegaFake_report.pdf")
md_content = converter.convert()
converter.save_md(md_content, Path("../megaparse/tests/output_tests/MegaFake_report_megaparse.md"))


### LLama Parse

In [20]:
from typing import List
from llama_index.core.schema import Document
import nest_asyncio

nest_asyncio.apply()
#GET LLAMA_CLOUD_API_KEY
import os
from llama_parse import LlamaParse
from llama_parse.utils import ResultType, Language

api_key: str | None = os.getenv("LLAMA_CLOUD_API_KEY")

parsing_instructions = "Do not take into account the page breaks (no --- between pages), do not repeat the header and the footer so the tables are merged. Keep the same format for similar tables."

parser = LlamaParse(
    api_key=str(api_key), 
    result_type=ResultType.MD,
    gpt4o_mode=True,
    verbose=True,
    language=Language.FRENCH,
    parsing_instruction=parsing_instructions,  # Optionally you can define a parsing instruction
)
# sync
documents: List[Document] = parser.load_data("../megaparse/tests/input_tests/MegaFake_report.pdf")

with open("../megaparse/tests/output_tests/MegaFake_report_llama.md", "w") as f:
        f.write(documents[0].get_content())


Started parsing the file under job_id 3fadd3c5-b647-49ec-8c73-413043bc7e9a


### Unstructured

In [22]:
from langchain_community.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("../megaparse/tests/input_tests/MegaFake_report.pdf", strategy="hi_res", infer_table_structure=True,
)
data = loader.load()

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
with open("../megaparse/tests/output_tests/MegaFake_report_unstructured.md", "w") as f:
        f.write(data[0].page_content)

### Evaluation with Diff Lib

In [16]:
import difflib

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.readlines()

def compare_files(source_path, target_path):
    source_lines = read_file(source_path)
    target_lines = read_file(target_path)

    diff = difflib.unified_diff(
    source_lines,
    target_lines,
    fromfile='target.md',
    tofile='generated.md',
    lineterm=''
    )

    modifications = 0
    for line in diff:
        #print(line)
        if line.startswith('+') and not line.startswith('+++'):
            modifications += 1
        elif line.startswith('-') and not line.startswith('---'):
            modifications += 1

    return modifications
    

diff_megaparse = compare_files("../megaparse/tests/output_tests/MegaFake_report_megaparse.md", "../megaparse/tests/output_tests/MegaFake_report.md")
diff_llamaparse = compare_files("../megaparse/tests/output_tests/MegaFake_report_llama.md", "../megaparse/tests/output_tests/MegaFake_report.md")
diff_unstructured = compare_files("../megaparse/tests/output_tests/MegaFake_report_unstructured.md", "../megaparse/tests/output_tests/MegaFake_report.md")

In [21]:

print(f"""Total number of modifications for llama parse: {diff_llamaparse}\nTotal number of modifications for megaparse: {diff_megaparse}\nTotal number of modifications for unstructured: {diff_unstructured}""")

Total number of modifications for llama parse: 31
Total number of modifications for megaparse: 26
Total number of modifications for unstructured: 93
